In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Veriyi yükle
df = pd.read_csv('../data/Spotify_dataset.csv')

# Tekrar eden track_id'leri temizle (Baseline'da yaptığımız gibi)
df = df.drop_duplicates(subset=['track_id']).reset_index(drop=True)

print(f"İşlenecek Veri Boyutu: {df.shape}")

İşlenecek Veri Boyutu: (89741, 21)


In [4]:
# Genre kolonunun adını kontrol edelim (track_genre veya genre olabilir)
genre_col = 'track_genre' if 'track_genre' in df.columns else 'genre'

# Çok fazla tür varsa, performansı düşürmemek için sadece en popüler 50 türü alabiliriz
# Ancak şimdilik tüm türleri dönüştürelim (RAM sorunu olursa burayı kısıtlarız)
print(f"Toplam benzersiz tür sayısı: {df[genre_col].nunique()}")

# One-Hot Encoding
# drop_first=True diyerek dummy variable trap'ten kaçınabiliriz ama 
# similarity tabanlı modellerde genelde hepsi tutulur.
genre_dummies = pd.get_dummies(df[genre_col], prefix='genre')

print(f"Oluşturulan genre feature sayısı: {genre_dummies.shape[1]}")

# Ana dataframe ile birleştirmeden önce RAM tasarrufu için tür verisini ayrı tutalım,
# finalde birleştireceğiz.

Toplam benzersiz tür sayısı: 113
Oluşturulan genre feature sayısı: 113


In [5]:
# Nümerik ses özellikleri
audio_cols = ['danceability', 'energy', 'loudness', 'speechiness', 
              'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

# MinMaxScaler: Veriyi 0-1 arasına çeker
scaler = MinMaxScaler()
df[audio_cols] = scaler.fit_transform(df[audio_cols])

print("Ses özellikleri 0-1 arasına ölçeklendi.")
print(df[audio_cols].head())

Ses özellikleri 0-1 arasına ölçeklendi.
   danceability  energy  loudness  speechiness  acousticness  \
0      0.686294  0.4610  0.791392     0.148187      0.032329   
1      0.426396  0.1660  0.597377     0.079067      0.927711   
2      0.444670  0.3590  0.736123     0.057720      0.210843   
3      0.270051  0.0596  0.573701     0.037617      0.908635   
4      0.627411  0.4430  0.737103     0.054508      0.470884   

   instrumentalness  liveness   valence     tempo  
0          0.000001    0.3580  0.718593  0.361245  
1          0.000006    0.1010  0.268342  0.318397  
2          0.000000    0.1170  0.120603  0.313643  
3          0.000071    0.1320  0.143719  0.746758  
4          0.000000    0.0829  0.167839  0.492863  


In [6]:
# Popülerlik Sınıflandırması (Segmentasyon)
def segment_popularity(popularity):
    if popularity >= 80:
        return 'High'
    elif popularity >= 50:
        return 'Medium'
    else:
        return 'Low'

df['popularity_segment'] = df['popularity'].apply(segment_popularity)

# Bunu da modele sokmak için One-Hot yapalım
pop_dummies = pd.get_dummies(df['popularity_segment'], prefix='pop_segment')
print("Popülerlik segmentleri oluşturuldu.")

Popülerlik segmentleri oluşturuldu.


In [7]:
# 1. Ses Özellikleri Matrisi
X_audio = df[audio_cols].values

# 2. Tür Matrisi
X_genre = genre_dummies.values

# 3. Popülerlik Matrisi
X_pop = pop_dummies.values

# Ağırlıklandırma (Opsiyonel ama Önemli):
# Tür (Genre) bilgisinin ses özelliklerinden daha baskın olmasını istiyorsak
# Genre matrisini bir katsayı ile çarpabiliriz. Şimdilik eşit bırakalım.
# X_genre = X_genre * 0.5 

# Tüm matrisleri yatayda (axis=1) birleştir
X_final = np.hstack([X_audio, X_genre, X_pop])

print(f"Final Feature Matrisi Boyutu: {X_final.shape}")

Final Feature Matrisi Boyutu: (89741, 125)


In [8]:
# İşlenmiş dataframe'i kaydet (Feature Engineering sonrası hali)
# Matris çok büyük olabileceği için dataframe'e geri dönüştürüp csv yapmak yerine 
# numpy array olarak kaydetmek daha performanslıdır ama 
# bootcamp projesinde CSV daha okunabilir olur.

# Hızlı erişim için metadata (Şarkı isimleri vb) + işlenmiş özellikleri birleştirelim
final_df = pd.concat([df[['track_id', 'track_name', 'artists', 'popularity']], 
                      pd.DataFrame(X_final)], axis=1)

# Dosya boyutu büyük olacağı için sadece gerekli kısmını veya pickle formatını kullanabiliriz
# Şimdilik clean_data olarak kaydedelim.
df.to_csv('../data/processed_data.csv', index=False)

# Final modelde kullanacağımız matrisi ayrı kaydedelim (Numpy formatında)
import joblib
joblib.dump(X_final, '../models/feature_matrix.pkl')
joblib.dump(scaler, '../models/scaler.pkl') # Scaler'ı da sakla, deployment'ta lazım olacak
joblib.dump(df['track_id'], '../models/track_ids.pkl') # ID eşleşmesi için

print("Veriler 'processed_data.csv' ve 'models/' klasörüne kaydedildi.")

Veriler 'processed_data.csv' ve 'models/' klasörüne kaydedildi.


## Feature Engineering Raporu

### Yapılan İşlemler
1.  **One-Hot Encoding (Genre):**
    * **Neden:** Baseline model, 'Metal' şarkısı ile hızlı bir 'Pop' şarkısını sadece tempoları aynı diye benzer sanıyordu.
    * **Çözüm:** Tür bilgisi matematiksel vektöre çevrildi. Artık model, öneri yaparken tür uyumuna da bakacak.
2.  **MinMax Scaling (Audio Features):**
    * **Neden:** `Loudness` (-60db) ve `Energy` (0-1) farklı ölçeklerdeydi. Genre verisi de 0-1 olduğu için tüm veriyi 0-1 aralığına çektik.
    * **Etki:** Kosinüs benzerliği hesaplarken tüm özellikler eşit ağırlığa sahip oldu.
3.  **Popularity Segmentation (Yeni Feature):**
    * **Türetilen:** `popularity` skoru 'Low', 'Medium', 'High' olarak kategorize edildi.
    * **Business Değeri:** Kullanıcıya "sadece popüler şarkıları öner" veya "keşfedilmemişleri öner" filtresi sunabilmemizi sağlayacak.